# Meta Training Experiment

In [1]:
import arrow
import learn2learn as l2l
import numpy as np
import os
import pickle
import torch
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt

from utils.evalUtils import print_confusion_matrix

from sklearn.preprocessing import MinMaxScaler

In [2]:
%run -i ./scripts/setConfigs.py

Set configs..


## Prepare data..

In [3]:
data_fn = os.path.join(data_path, 'simulation_data_y_2020_2021_reduced.h5')
df_data_train = pd.read_hdf(data_fn, key='df')
print('Shape of X_train data: {}'.format(df_data_train.shape))

Shape of X_train data: (105216, 17)


In [4]:
anomalous_data_fn = os.path.join(data_path, 'anomalous_data_y_2022_reduced.h5')
df_anormal = pd.read_hdf(anomalous_data_fn, key='df')
print('Shape of X_anomal aka X_test: {}'.format(df_anormal.shape))

Shape of X_anomal aka X_test: (35040, 18)


In [5]:
drifted_data_fn = os.path.join(data_path, 'drifted_data_y_2023_reduced_more_cos_phi.h5')
df_x_drifted = pd.read_hdf(drifted_data_fn, key='df')
print('Shape of X_drifted: {}'.format(df_x_drifted.shape))

Shape of X_drifted: (35040, 18)


In [6]:
anomalous_drifted_data_fn = os.path.join(data_path, 'anomalous_drifted_data_y_2023_reduced_more_cos_phi.h5')
df_anormal_drifted = pd.read_hdf(anomalous_drifted_data_fn, key='df')
print('Shape of X_anomal,drifted : {}'.format(df_anormal_drifted.shape))

Shape of X_anomal,drifted : (35040, 19)


In [7]:
y_anormal = [1 if x > 0 else 0 for x in df_anormal['label']]
df_anormal.drop('label', inplace=True, axis=1)
x_anormal = df_anormal

In [9]:
s_x_drifted_drift_labels = df_x_drifted['drift_labels']
df_x_drifted.drop('drift_labels',axis=1, inplace=True)

In [10]:
y_anormal_drifted = [1 if x > 0 else 0 for x in df_anormal_drifted['anomaly_labels']]
df_anormal_drifted.drop(['anomaly_labels', 'drift_labels'], inplace=True, axis=1)
x_anormal_drifted = df_anormal_drifted

In [9]:
print('Scale data..')
scaler_train = MinMaxScaler((-1,1))
scaler_train = scaler_train.fit(df_data_train)
scaled_train = scaler_train.transform(df_data_train.to_numpy())
x_anormal = scaler_train.transform(x_anormal.to_numpy())
x_anormal_drifted = scaler_train.transform(x_anormal_drifted.to_numpy())

Scale data..


In [10]:
X_train, X_test, y_train, y_test = train_test_split(scaled_train, fake_labels, train_size=0.8, random_state=42)

In [11]:
assert len(scaled_train) == len(X_test) + len(X_train)
assert len(scaled_train) == len(y_test) + len(y_train)

In [12]:
# build tensor from numpy
train_torch_tensor_X = torch.from_numpy(X_train).type(torch.FloatTensor)
train_torch_tensor_y = torch.from_numpy(y_train.reshape(len(y_train),1)).type(torch.FloatTensor)

test_torch_tensor_X = torch.from_numpy(X_test).type(torch.FloatTensor)
test_torch_tensor_y = torch.from_numpy(y_test.reshape(len(y_test),1)).type(torch.FloatTensor)

In [13]:
x_anormal_torch_tensor = torch.from_numpy(x_anormal).type(torch.FloatTensor)
x_anormal_drifted_torch_tensor = torch.from_numpy(x_anormal_drifted).type(torch.FloatTensor)

In [14]:
# build pytorch dataset from tensor
train_dataset = TensorDataset(train_torch_tensor_X, train_torch_tensor_y)
test_dataset = TensorDataset(test_torch_tensor_X, test_torch_tensor_y)

In [15]:
# create MetaDatasets from l2l
train_metadataset = l2l.data.MetaDataset(train_dataset)
test_metadataset = l2l.data.MetaDataset(test_dataset)

## Import model..

In [16]:
%run -i ./scripts/ReadSimpleAE.py

++++++++++++++++++++++++++++++
Load untrained simple AE..
----------
Load model: /home/torge/dev/masterthesis_code/02_Experimente/03_Experimente/models/model_bib/20200302_firstAE_model.pt
----------
SimpleAutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=17, out_features=12, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=12, out_features=8, bias=True)
    (3): Tanh()
  )
  (decoder): Sequential(
    (0): Linear(in_features=8, out_features=12, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=12, out_features=17, bias=True)
    (3): Tanh()
  )
)
----------
Init weights..
----------
Set model in train mode!
++++++++++++++++++++++++++++++


## Read sampled data!

In [17]:
fn = '20200407_teilfaktorieller_versuchsplan_gesampelt.csv'
df_versuchsplan = pd.read_csv(os.path.join(exp_data_path, fn), sep=';')

In [18]:
len(df_versuchsplan)

97

## Start!

In [19]:
from Experiments.MetaTrainExperiment import MetaTrainExperiment

In [20]:
optimizer = None
adaption_steps = None
k_train = None
num_train = None
num_eval = None
num_iter = None
meta_lr = None
first_order = None

In [21]:
for i , val in df_versuchsplan.iterrows():
    print('Current: {} of {}'.format(i, len(df_versuchsplan)))
    k_train = val['k_train']
    adaption_steps = val['adaption_steps']
    optimizer = val['optimizer']
    num_train = val['num_train']
    num_eval = val['num_eval']
    num_iter = val['num_iter']
    meta_lr = val['meta_lr']
    first_order = True
    print(val)
    
    model_fn = '{}_MODEL__K_TRAIN_{}_NUM_TRAIN_{}_NUM_EVAL_{}_NUM_ITER_{}_OPTIMIZER_{}_META_LR_{}_ADAPTION_STEPS_{}.pt'.format(arrow.now().format('YYYYMMDD'), 
                                                                                                                    k_train, num_train, num_eval,
                                                                                                                   num_iter, optimizer, meta_lr, adaption_steps)

    model_fn = os.path.join(exp_models_path, model_fn)

    logreg_fn = '{}_LOGREG_MODEL__K_TRAIN_{}_NUM_TRAIN_{}_NUM_EVAL_{}_NUM_ITER_{}_OPTIMIZER_{}_META_LR_{}_ADAPTION_STEPS_{}.save'.format(arrow.now().format('YYYYMMDD'), 
                                                                                                                    k_train, num_train, num_eval,
                                                                                                                   num_iter, optimizer, meta_lr, adaption_steps)

    logreg_fn = os.path.join(exp_models_path, logreg_fn)
    
    
    exp_name = '{}_EXPERIMENT__K_TRAIN_{}_NUM_TRAIN_{}_NUM_EVAL_{}_NUM_ITER_{}_OPTIMIZER_{}_META_LR_{}_ADAPTION_STEPS_{}.csv'.format(arrow.now().format('YYYYMMDD'), 
                                                                                                                    k_train, num_train, num_eval,
                                                                                                                   num_iter, optimizer, meta_lr, adaption_steps)
    exp_name = os.path.join(exp_data_path, exp_name)
    
    
    meta_train_exp = MetaTrainExperiment(model=model, train_data=train_metadataset, eval_data=test_metadataset, 
                                     optimizer=optimizer, meta_lr=meta_lr, num_iterations=num_iter, k_train=k_train,
                                    num_train=num_train, num_eval=num_eval, adaption_steps=adaption_steps,
                                    x_test= x_anormal_torch_tensor, ano_label_x_test=y_anormal, x_drifted_ano=x_anormal_drifted_torch_tensor, 
                                    ano_label_x_drifted_ano= y_anormal_drifted, model_fn=model_fn, logreg_fn=logreg_fn, exp_name=exp_name)
    
    meta_train_exp.run()
    print('+++' * 10)

  0%|          | 0/300 [00:00<?, ?it/s]

Current: 0 of 97
k_train            10
num_train         128
num_eval           16
num_iter          300
optimizer         SGD
meta_lr           0.1
adaption_steps      5
Name: 0, dtype: object


Adaption Error: 0.049935, Validation Error: 0.028072: 100%|██████████| 300/300 [09:45<00:00,  1.95s/it]
  0%|          | 0/100 [00:00<?, ?it/s]

Saved model at: /home/torge/dev/masterthesis_code/02_Experimente/03_Experimente/exp_models/20200407_MODEL__K_TRAIN_10_NUM_TRAIN_128_NUM_EVAL_16_NUM_ITER_300_OPTIMIZER_SGD_META_LR_0.1_ADAPTION_STEPS_5.pt
Saved logreg at: /home/torge/dev/masterthesis_code/02_Experimente/03_Experimente/exp_models/20200407_LOGREG_MODEL__K_TRAIN_10_NUM_TRAIN_128_NUM_EVAL_16_NUM_ITER_300_OPTIMIZER_SGD_META_LR_0.1_ADAPTION_STEPS_5.save
++++++++++++++++++++++++++++++
Current: 1 of 97
k_train              10
num_train           128
num_eval             32
num_iter            100
optimizer          Adam
meta_lr           0.001
adaption_steps       10
Name: 1, dtype: object


Adaption Error: 0.007878, Validation Error: 0.007504: 100%|██████████| 100/100 [05:43<00:00,  3.43s/it]
  0%|          | 0/500 [00:00<?, ?it/s]

...

Saved model at: /home/torge/dev/masterthesis_code/02_Experimente/03_Experimente/exp_models/20200408_MODEL__K_TRAIN_20_NUM_TRAIN_128_NUM_EVAL_16_NUM_ITER_300_OPTIMIZER_SGD_META_LR_0.01_ADAPTION_STEPS_10.pt
Saved logreg at: /home/torge/dev/masterthesis_code/02_Experimente/03_Experimente/exp_models/20200408_LOGREG_MODEL__K_TRAIN_20_NUM_TRAIN_128_NUM_EVAL_16_NUM_ITER_300_OPTIMIZER_SGD_META_LR_0.01_ADAPTION_STEPS_10.save
++++++++++++++++++++++++++++++
Current: 95 of 97
k_train              5
num_train           32
num_eval            32
num_iter           100
optimizer          SGD
meta_lr           0.01
adaption_steps      10
Name: 95, dtype: object


Adaption Error: 0.000380, Validation Error: 0.000598: 100%|██████████| 100/100 [01:41<00:00,  1.02s/it]
  0%|          | 0/100 [00:00<?, ?it/s]

Saved model at: /home/torge/dev/masterthesis_code/02_Experimente/03_Experimente/exp_models/20200408_MODEL__K_TRAIN_5_NUM_TRAIN_32_NUM_EVAL_32_NUM_ITER_100_OPTIMIZER_SGD_META_LR_0.01_ADAPTION_STEPS_10.pt
Saved logreg at: /home/torge/dev/masterthesis_code/02_Experimente/03_Experimente/exp_models/20200408_LOGREG_MODEL__K_TRAIN_5_NUM_TRAIN_32_NUM_EVAL_32_NUM_ITER_100_OPTIMIZER_SGD_META_LR_0.01_ADAPTION_STEPS_10.save
++++++++++++++++++++++++++++++
Current: 96 of 97
k_train             10
num_train           32
num_eval            16
num_iter           100
optimizer         Adam
meta_lr           0.01
adaption_steps       5
Name: 96, dtype: object


Adaption Error: 0.004397, Validation Error: 0.000122: 100%|██████████| 100/100 [01:01<00:00,  1.62it/s]


Saved model at: /home/torge/dev/masterthesis_code/02_Experimente/03_Experimente/exp_models/20200408_MODEL__K_TRAIN_10_NUM_TRAIN_32_NUM_EVAL_16_NUM_ITER_100_OPTIMIZER_Adam_META_LR_0.01_ADAPTION_STEPS_5.pt
Saved logreg at: /home/torge/dev/masterthesis_code/02_Experimente/03_Experimente/exp_models/20200408_LOGREG_MODEL__K_TRAIN_10_NUM_TRAIN_32_NUM_EVAL_16_NUM_ITER_100_OPTIMIZER_Adam_META_LR_0.01_ADAPTION_STEPS_5.save
++++++++++++++++++++++++++++++
